In [ ]:
import numpy as np
import pandas as pd
import sys
from IPython.display import display

def data_cleaner():
    print("Data Cleaner v1.0 by Kevin Cho, Ph.D.")
    print("Currently allows for automated changes to tables that do not contain any NULL values.")
    print("Upcoming v1.1 update:")
    print("1) Imputes data by deleting data points containing NULL values before changing values")
    print("2) Export to .csv or MySQL database")
    print("3) Menu interface for repeating functions (ex: renaming columns)")
    print("4) Summarizing data by mean, mode, std dev, etc")
    print("5) Recommend machine learning technique for classification based on accuracy in cross validation")
    print("")
    print("")
    print("Please make sure the following are imported:")
    print("import numpy as np")
    print("import pandas as pd")
    print("import sys")
    print("from IPython.display import display")
    print("")
    
    #Ensure input is placed below print as problems have arisen in jupyter notebook
    sys.stdout.flush()
    
    #Step 1: Get data
    data_url = (input("Enter the URL of the dataset to be cleaned: "))
    print("")
    dataset = pd.read_csv(data_url)
    print("Table contains {} columns...preparing for column naming".format(len(dataset.columns)))
    
    #Step 2: User input data column renaming
    dataset = col_rename(dataset)
    
    #Step 3A: Create column dictionary and save original
    data_col_dict = col_dict(dataset)
    
    #Step 3B: Create an original copy of the data column dictionary for reference
    original_data_col_dict = dict(data_col_dict)
    
    #Step 4: Run function to view data 1:15 separate in order for calling later
    view_dataset(dataset)  
    print("Displaying dictionary of column number : named column.")
    print(data_col_dict)
    
    #Step 5A: Create empty dataframe to replace NaN values
    df_build = create_dict_df(dataset, data_col_dict)
    
    #Step 5B: Summarize data
    sum_df = collect_col_data(dataset, df_build, data_col_dict)
    print("")
    print("Printing count of values in dataset...")
    print("")
    
    #Step 5C: Creating a copy of the original summary
    original_sum_df = pd.DataFrame(sum_df)
    
    #Step 5D: Displaying the summary of dataset split in 2 for easier viewing
    sum_df_1half = sum_df.ix[:,0:len(dataset.columns)+1]
    sum_df_2half = sum_df.ix[:,len(dataset.columns)+1:(len(dataset.columns)*2)]
    display(sum_df_1half)
    print("")
    display(sum_df_2half)
    
    #Step 6: Changing column values
    dataset, convert_key = convert_values(dataset, data_col_dict)
    print("Data has been prepared for quantitative analysis")
    print("Displaying old keys and new keys...")
    display(convert_key)
    print("Displaying first 15 items in dataset...")
    display(dataset.head(15))
    

    
    
#Function: step 2 - column rename
def col_rename(dataset):
    data_columns = []
    #Consider having input split
    for i in range(0,len(dataset.columns)):
        data_columns.append(str(input("Enter the name of column {}: ".format(i+1))))
    print("")
    dataset.columns = data_columns
    return dataset

#Function: step 3A - column dictionary
def col_dict(dataset):
    data_col_dict = {n+1: list(dataset.columns)[n] for n in range(0,len(list(dataset.columns)))}
    return data_col_dict

#Function: step 4 - viewing dataset
def view_dataset(dataset):
    #Next implementation: ask user for head, slice, tail -> single number for head/tail and x:y for slice
    view_yes = input("Would you like to view this dataset (Y/N)? ")
    while view_yes not in ['Y', 'N']:
        print("Could not understand, please use either 'Y' or 'N'.")
        print("")
        sys.stdout.flush()
        view_yes = input("Would you like to view this dataset (Y/N)? ")
    if view_yes == 'Y':
            print("")
            display(dataset.head(15))
            print("")
    elif view_yes == 'N':
            print ("")
            
#Function: step 4.5 - scanning column for names and counts
def scan_values(dataset, data_dict, n):
    count = 0
    naming = []
    name_count = []
    for name in dataset[data_dict[n+1]]:
        if name not in naming:
            naming.append(name)
            name_count.append(1)
        else:
            name_count[naming.index(name)]+=1
    return naming, name_count

#Function: step 5A - creating dataframe for summary
def create_dict_df(dataset, data_dict):
    data_col_list = list(dataset.columns)
    data_col_list_exp = []
    for p in range(0, len(data_col_list)):
        data_col_list_exp.append(data_col_list[p] + " value")
        data_col_list_exp.append(data_col_list[p] + " count")
    for i in range(0,len(dataset.columns)):
        naming, name_count = scan_values(dataset, data_dict, i)
        max_col_len = 0
        if max_col_len < len(naming):
                max_col_len = len(naming)
    df_info = pd.DataFrame(np.nan, index=range(1,max_col_len+1), columns = data_col_list_exp)
    return df_info
            
#Function: step 5B - view data by slices
def collect_col_data(dataset, df_build, data_dict):
    print("")
    print("DISCLAIMER: Function does not sort by levels, be mindful when entering converted stats")
    print("DISCLAIMER: Currently a function to impute data does not exist and awaits future implementation")
    print("")
    #Ensure input is placed below print as problems have arisen in jupyter notebook
    sys.stdout.flush()
    df_col_list = list(dataset.columns)
    summary_df = pd.DataFrame(df_build)
    for i in range(0,len(df_col_list)):
        naming, name_count = scan_values(dataset, data_dict, i)
        while len(naming)<4:
            naming.append(" ")
        while len(name_count) < 4:
            name_count.append(" ")
        #dataset[data_col_dict[i]].replace(name, new_name, inplace=True)
        for j in summary_df.index.values:
            if i == 0:
                summary_df.ix[j,i] = naming[j-1]
                summary_df.ix[j,i+1] = name_count[j-1]
            else:
                summary_df.ix[j,i*2] = naming[j-1]
                summary_df.ix[j,(i*2)+1] = name_count[j-1]
    return summary_df

#Step 6: Change column data after user chooses to do so or not
def convert_values(dataset, data_col_dict):
    #Create a dataframe to record old/new names
    convert_key = pd.DataFrame(np.nan, index=dataset.columns, columns = ['old key', 'new key'])
    convert_key = convert_key.astype(object)
    for i in range(0,len(dataset.columns)):
        print("")
        print("Changing values in '{}'. ".format(data_col_dict[i+1]))
        naming, name_count = scan_values(dataset, data_col_dict, i)
        print("Current values are {}.".format(naming))
        print("")
        sys.stdout.flush()
        choice = input("Do you want to rename values for this? (answer with 'Y' or 'N') ")
        while choice not in ['Y', 'N']:
            print("Could not understand, please use either 'Y' or 'N'.")
            choice = input("Do you want to rename values for this? (answer with 'Y' or 'N')")
        if choice == 'Y':
            new_name = [x for x in
                input("Enter the values in order and spaced as floats to replace the values listed above: ").split()]
            #Add old/new to dictionary for convert_key
            convert_key.ix[data_col_dict[i+1],'old key'] = list(naming)
            convert_key.ix[data_col_dict[i+1],'new key'] = list(new_name)
            for j in range(0,len(naming)):
                dataset[data_col_dict[i+1]].replace(naming[j], new_name[j], inplace=True)
        elif choice == 'N':
            print("Skipping renaming '{}'.".format(data_col_dict[i+1]))
            convert_key.ix[data_col_dict[i+1],'old key'] = list(naming)
            convert_key.ix[data_col_dict[i+1],'new key'] = 'Unchanged'
    return dataset, convert_key
            
data_cleaner()